## Step 1 - Start local geth node


In [90]:
# Step 1 - Start local geth node
# Run runGeth script    
# $ ./00_runGeth.sh
# 
# Or the following but remember to include the preloaded keyfiles for all actors 
# geth --dev --dev.period 1 --datadir ./testchain --rpc --rpccorsdomain '*' --rpcport 8646 --rpcapi "eth,net,web3,debug" --port 32323 --maxpeers 0 --targetgaslimit 994712388 console

In [28]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import web3, solc, json
from web3.contract import ConciseContract
from web3 import Web3
from web3.auto.gethdev import w3

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
# ipc:./testchain/geth.ipc

w3 = Web3(Web3.IPCProvider("./testchain/geth.ipc"))

In [30]:
from web3.middleware import geth_poa_middleware
w3.middleware_stack.inject(geth_poa_middleware, layer=0)

In [31]:
# Set Up Environment

In [32]:
# Check that our geth node is running
block = w3.eth.getBlock('latest')
block.number

3

In [33]:
block.hash


HexBytes('0xa53b875438060c76116ea10cd9c1fd8d1da5bee2604c3f0e7138a4e07d17f8c4')

In [34]:
# Checks that test accounts have loaded correctly. 
w3.eth.accounts[2]

# >'0xa22AB8A9D641CE77e06D98b7D7065d324D3d6976'

'0xa22AB8A9D641CE77e06D98b7D7065d324D3d6976'

## Step 2 - Set Up Environment

In [35]:
# compile contract, check which version. ie solidity 0.4.25
# !pip install solc
!solc --version

solc, the solidity compiler commandline interface
Version: 0.4.25+commit.59dbf8f1.Darwin.appleclang


In [36]:
# load web3 python helper functions 
SCRIPTSDIR="scripts"
!cp $SCRIPTSDIR/* .
import functions

In [49]:
# File operations for solidity flattener
!chmod 755 ./scripts/solidityFlattener.pl
!mkdir flattened

mkdir: flattened: File exists


In [45]:
# Unlock user accounts - Takes a few seconds
functions.unlockAccounts('');

In [46]:
# Test helper fucntions 
functions.printBalances()

 # Account                                      EtherBalanceChange                   Token Name
-- ------------------------------------------ -------------------- ----------------------- ---------------------
 1 0xa11AAE29840fBb5c86E6fd4cF809EBA183AEf433              1000000                       0 Account #1 - Owner
 2 0xa22AB8A9D641CE77e06D98b7D7065d324D3d6976              1000000                       0 Account #2 - Alice
 3 0xa33a6c312D9aD0E0F2E95541BeED0Cc081621fd0              1000000                       0 Account #3 - Bob
 4 0xa44a08d3F6933c69212114bb66E2Df1813651844              1000000                       0 Account #4 - Carol
 5 0xa55A151Eb00fded1634D27D1127b4bE4627079EA              1000000                       0 Account #5 - Dave
 6 0xa66a85ede0CBE03694AA9d9dE0BB19c99ff55bD9              1000000                       0 Account #6
 7 0xa77A2b9D4B1c010A22A7c565Dc418cef683DbceC              1000000                       0 Account #7
 8 0xA88A05d2b88283ce84C8325760B72a64591

## Step 3 - Deploy Base Smart Contracts

In [47]:
# define
def sign_transaction(txn):
    # Flesh out the transaction for local signing
    next_nonce = w3.eth.getTransactionCount(str(w3.eth.accounts[0]))
    signable_transaction = dict(
      txn,
      nonce=next_nonce,
      gasPrice=w3.toWei(4, 'gwei'),
    )
    # Sign transaction
    encrypted_key = open('./testchain/keystore/UTC--2017-05-20T02-37-30.360937280Z--a00af22d07c87d96eeeb0ed583f8f6ac7812827e').read()
    private_key = w3.eth.account.decrypt(encrypted_key, '')
    signature_info = w3.eth.account.signTransaction(signable_transaction, private_key)
    # Broadcast transaction
    txn_hash = w3.eth.sendRawTransaction(signature_info.rawTransaction)
    # Wait for the transaction to be mined
    receipt = w3.eth.waitForTransactionReceipt(txn_hash)

    return receipt

In [88]:
## Warning, this is for testing purposes only, this is not a secure way to handle private key. Needs to be rewritten.

### Step 3a - Contract Deployment

#### Build Contract

In [59]:
WHITELISTSOL = "./flattened/WhiteList_flattened.sol"
!./scripts/solidityFlattener.pl --contractsdir=./contracts --mainsol=WhiteList.sol --outputsol={WHITELISTSOL} --verbose

contractsdir: ./contracts
remapdir    : (no remapping)
mainsol     : WhiteList.sol
outputsol   : ./flattened/WhiteList_flattened.sol
Processing contracts/WhiteList.sol
    Importing contracts/Controlled.sol
    Processing contracts/Controlled.sol
        Importing contracts/Owned.sol
        Processing contracts/Owned.sol
    Importing contracts/WhiteListInterface.sol
    Processing contracts/WhiteListInterface.sol


In [60]:
WhiteListOutput = !solc --optimize --combined-json abi,bin,interface {WHITELISTSOL}
i = 0
while (i < len(WhiteListOutput)):
    if WhiteListOutput[i][:10] == '{"contract':
        print(WhiteListOutput[i][:20])
        WhiteListOutputContract = WhiteListOutput[i]
    i = i + 1

WhiteListAbi = json.loads(WhiteListOutputContract)['contracts'][WHITELISTSOL + ':WhiteList']['abi'] 
WhiteListBin = json.loads(WhiteListOutputContract)['contracts'][WHITELISTSOL + ':WhiteList']['bin']

# Build the contract object in preparation for deployment


{"contracts":{"./fla


In [ ]:
#WhiteListBin

In [89]:
WhiteListAbi

#### Deploy WhiteList

In [65]:
## Deploy WhiteList
whitelist_deployer = w3.eth.contract(abi=WhiteListAbi, bytecode=WhiteListBin)
init = whitelist_deployer.constructor()
txn = init.buildTransaction({'gas': 1000000})
receipt = sign_transaction(txn)
WhiteListAddress = receipt.contractAddress
WhiteListAddress

'0x003575135a878273fcdd691c446e7B75fdECF9eC'

In [66]:
# Build ConciseContract artifact
whitelist_contract = w3.eth.contract(
    address=WhiteListAddress,
    abi=WhiteListAbi,
)
whitelist_CContract = ConciseContract(whitelist_contract)
whitelist_CContract

## Step 4 - Interacting with Smart Contract 

In [78]:
print("WhiteListAddress:             " + WhiteListAddress + "    Deployed:" + str(whitelist_CContract.isOwner()) )

WhiteListAddress:             0x003575135a878273fcdd691c446e7B75fdECF9eC    Deployed:True


#### 4a - Add accounts to whitelist

In [75]:
# Whitelist 
insert_list = [w3.eth.accounts[2], w3.eth.accounts[3]]

# Submit txn
txn = whitelist_contract.functions.add(insert_list).buildTransaction({'gas': 320000})
receipt = sign_transaction(txn)
receipt.status

1

In [76]:
# Using the standard web3 contract call 
dst_name = whitelist_contract.functions.isInWhiteList(w3.eth.accounts[3]).call()
dst_name

True

In [77]:
# Using the ConciseContract call 
whitelist_CContract.isInWhiteList(w3.eth.accounts[3])

True

#### 4b - Checking Controller

In [79]:
whitelist_CContract.isControllable()

True

In [87]:
# Submit txn to add new controller "accounts[5]"
txn = whitelist_contract.functions.addController(w3.eth.accounts[3]).buildTransaction({'gas': 320000})
receipt = sign_transaction(txn)
receipt.status

0

## Misc Code Blocks

In [82]:
## Get more information from a transaction hash
w3.eth.getTransaction(receipt.transactionHash)

AttributeDict({'blockHash': HexBytes('0xa4f05cd6586ae67cfb27e7bbbeea30430a784abafaeb9ef6bd1e44bc5da1db02'),
 'blockNumber': 507,
 'from': '0xa00Af22D07c87d96EeeB0Ed583f8F6AC7812827E',
 'gas': 320000,
 'gasPrice': 4000000000,
 'hash': HexBytes('0x027e61b2223cb7ddd4cccd1f1c063042129ea42116237e9506f2e4dbb1381214'),
 'input': '0xc4c1c94f00000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000002000000000000000000000000a22ab8a9d641ce77e06d98b7d7065d324d3d6976000000000000000000000000a33a6c312d9ad0e0f2e95541beed0cc081621fd0',
 'nonce': 13,
 'to': '0x003575135a878273fcdd691c446e7B75fdECF9eC',
 'transactionIndex': 0,
 'value': 0,
 'v': 27,
 'r': HexBytes('0xe83351c4b179220c5314c61b2582dc27fb9df815188dcb998f437f8b62bc46e2'),
 's': HexBytes('0xd33fa0d94b137e2d53dfa001786bd22b50cdc9b045b5a14092146888034cdb')})

In [83]:
## Get the contracts final memory from a transaction receipt
w3.providers[0].make_request("debug_traceTransaction", [str(ascii(receipt.transactionHash))[10:-2], {}])['result']['structLogs'][-1]['memory']

['000000000000000000000000a33a6c312d9ad0e0f2e95541beed0cc081621fd0',
 '0000000000000000000000000000000000000000000000000000000000000002',
 '00000000000000000000000000000000000000000000000000000000000000e0',
 '0000000000000000000000000000000000000000000000000000000000000000',
 '0000000000000000000000000000000000000000000000000000000000000002',
 '000000000000000000000000a22ab8a9d641ce77e06d98b7d7065d324d3d6976',
 '000000000000000000000000a33a6c312d9ad0e0f2e95541beed0cc081621fd0',
 '0000000000000000000000000000000000000000000000000000000000000001']